**Текущие задачи**:
- Фактически для проверки начала работы, нам достаточно знать лишь `parse_time`
- Выгружать датасет из памяти после того, как он был считан. Составлять новый датасет, выводить в csv и конкатенировать сами csv.
- Функция обновления views_num (и других показателей)
- Функция поиска новых статей и составления по ним записей: сравниваем текущую дату и дату последней статьи, запускаем поочередное обращение к ресурсам
- Оформить корпус: csv + zip с файлами полных текстов
- Однотипное оформление столбцов
- Однотипное оформление файлов

# Парсер

Реализованы парсеры данных и собраны тексты и метаинформация статей следующих ресурсов:
- Хабрахабр https://habr.com/
- Типичный программист https://tproger.ru/

Исходные данные собраны посредством `wget` и уже предобработаны. Однако со временем на ресурсах публикуются новые статьи, а старые статьи продолжают набирать просмотры. Сбор и обновление соответствующей информации помогают учесть появление новых технологий.

Чтобы не создавать повышенную нагрузку на серверы ресурсов и не вызывать их подозрений, необходимо их опрашивать поочередно. Все необходимые методы содержатся внутри класса `Parser`, а оркестровку парсеров проводит объект класса `ParserComposer`.

In [1]:
# standard library
import datetime
import zipfile
import re
import uuid

# conections
import requests
from requests.exceptions import ConnectionError

# tor network for private parsing with privoxy and stem
# https://gist.github.com/KhepryQuixote/46cf4f3b999d7f658853
from stem import Signal
from stem.control import Controller

# for tor network IP changing
with Controller.from_port(port = 9051) as controller:
    controller.authenticate('07011951')
    controller.signal(Signal.NEWNYM)

headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.73.11 (KHTML, like Gecko) Version/7.0.1 Safari/537.73.11'
}

# parsers libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# data science 
import pandas as pd
from tqdm.notebook import tqdm, trange

sources = {
    'tproger':
        {
            'base_url': 'https://tproger.ru/',
            'first_page_url': 'https://tproger.ru/page/1/',
            'find_all_args': ('a', {'class':'article-link'})
        },
    'habr':
        {
            'base_url': 'https://habr.com/',
            'first_page_url': 'https://habr.com/ru/page1/',
            'find_all_args': ('a', {'class':'post__title_link'})
        }
}

for source in sources:
    sources[source]['df'] = pd.read_csv(f'../../DATASETS/{source}.csv.zip',
                               index_col=0,
                               parse_dates=['post_time', 'parse_time'])

In [2]:
class Parser:
    def __init__(self, source_name):
        self.source_name = source_name
        self.source = sources[source_name]
        self.df = pd.read_csv(f'../../DATASETS/{source_name}.csv.zip',
                         index_col=0,
                         parse_dates=['post_time', 'parse_time'])
    
    def delta_dates_days(self):
        today_date = datetime.datetime.now()
        last_parse_date = self.df.post_time.max()
        today_date = today_date.replace(tzinfo=last_parse_date.tz)
        delta_dates = today_date - last_parse_date
        return delta_dates.days
    
    def check_new_articles(self):
        '''Проверяет, появились ли на ресурсе новые публикации,
        и если появились, заносит их в план на скачивание.'''
        all_new_urls = set()
        page_count = 1
        source = self.source
        proxies = {
            'http': 'socks5h://127.0.0.1:9050',
            'https': 'socks5h://127.0.0.1:9050'
        }
        while True:
            try:
                page_url = source['first_page_url'].replace('1', str(page_count))
                page = requests.get(page_url,
                                    headers=headers,
                                    proxies=proxies,
                                    stream=False)
                soup = BeautifulSoup(page.text, 'html.parser')
                urls = {url['href'] for url in soup.find_all(*source['find_all_args'])}
                if not urls:
                    # the thread processes the case when captcha and no urls
                    print("Changing proxy because of no urls or CAPTCHA...")
                    proxies = None
                    continue
                old_urls = set(source['df'].index.to_list())
                new_urls = urls - old_urls
                all_new_urls = all_new_urls | new_urls
                self.new_articles_urls = all_new_urls
                print(f"{len(all_new_urls)} urls are collected from {source['base_url']}")
                if (len(new_urls) < len(urls)) or (len(urls) == 0):
                    print(f"{len(all_new_urls)} new urls are saved for {source['base_url']}.")
                    break
                else:
                    page_count += 1
            except ConnectionError:
                print("Problems with connection...")
                continue
                
        def add_full_text(self, full_text):
            zpath = f"../../DATASETS/{self.source_name}.zip"
            with zipfile.ZipFile(zpath, 'a') as zipped_f:
                full_text = re.sub('\n+', '\n\n', full_text).strip()
                zipped_f.writestr(f"{self.source_name}/{data['filename']}", full_text)
                add_full_text
    def concat_csv(self):
        '''
        Соединяет старый csv и тот, что получен из новых данных.
        '''
        pass
    

class HabrParser(Parser):
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        data['title'] = soup.select_one('h1').text.strip()
        date_time = soup.select_one('.post__time').get('data-time_published')
        data['post_time'] = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%MZ')
        data['views_num'] = soup.select_one('.post-stats__views-count').text.strip()
        data['likes_num'] = int(soup.select_one('.js-post-vote').text.strip().replace('–', '-'))
        data['favs_num'] = int(soup.select_one('.bookmark__counter').text)
        data['comments_num'] = int(soup.select_one('.post-stats__comments-count').text)
        data['filename'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, url))
        full_text = page_soup.select_one('.post__body_full').text
        if url not in self.df:
            self.add_full_text(full_text)
        return data


class ParserComposer:
    def __init__(self):
        pass
    
    def scheduler(self):
        '''
        Сортирует задачи таким образом, чтобы
        равномерно распределить запросы к внешним ресурсам.
        Порядок проверки:
        1) новые статьи
        2) обновление просмотров статей, вышедших в последний месяц (post_time)
        3) обновление просмотров статей, проверявшихся > месяца назад (parse_time), не относящихся к п. 2
        '''
        pass
    
    def pipeline(self):
        '''Проверяем, вышли ли новые статьи. Планируем загрузки. Обновляем датасеты.'''
        pass

In [3]:
habr_parser = HabrParser('habr')
# habr_parser.check_new_articles()

In [4]:
tproger_parser = TprogerParser('tproger')
#tproger_parser.check_new_articles()

In [146]:
data

{'title': 'Rust 1.51.0: const generics MVP, новый распознаватель функциональности Cargo',
 'post_time': datetime.datetime(2021, 3, 26, 17, 8),
 'views_num': '514',
 'likes_num': 8,
 'favs_num': 3,
 'comments_num': 4,
 'filename': '9db42ff8-c51d-58c5-813d-ff5c8bddb30e'}